In [11]:
import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import os

In [8]:
num_classes = 81
img_rows, img_cols = 32,32
batch_size = 16

train_data_dir = './Data/Training/'
validation_data_dir = './Data/Test/'

train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  rotation_range = 30,
                                  width_shift_range = .3,
                                  height_shift_range = .3,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_rows, img_cols),
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size = (img_rows, img_cols),
                                                              batch_size = batch_size,
                                                              class_mode = 'categorical',
                                                              shuffle = False)

Found 41322 images belonging to 81 classes.
Found 13877 images belonging to 81 classes.


In [12]:
input_shape = (img_rows, img_cols, 3)

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_3 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 64)       

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop, SGD

checkpoint = ModelCheckpoint('./Model/fruits_fresh_cnn_1.h5',
                             monitor = 'val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta= 0,
                         patience = 3,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=.2,
                             patience=3,
                             verboise = 1,
                             min_delta = .0001)

callbacks = [earlystop, checkpoint]

model.compile(loss='categorical_crossentropy',
             optimizer = RMSprop(lr=.001),
             metrics=['accuracy'])

no_train = 41322
no_val = 13877
epochs = 5

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = no_train // batch_size,
                    epochs  = epochs,
                    callbacks = callbacks,
                    validation_data = validation_generator,
                    validation_steps=no_val // batch_size)

Epoch 1/5
2582/2582 [==============================] - 896s 346ms/step - loss: 2.8111 - accuracy: 0.2510 - val_loss: 0.4744 - val_accuracy: 0.8501

Epoch 00001: val_loss improved from inf to 0.47445, saving model to ./Model\fruits_fresh_cnn_1.h5
Epoch 2/5
2582/2582 [==============================] - 145s 56ms/step - loss: 0.7743 - accuracy: 0.7519 - val_loss: 0.2919 - val_accuracy: 0.9038

Epoch 00002: val_loss improved from 0.47445 to 0.29193, saving model to ./Model\fruits_fresh_cnn_1.h5
Epoch 3/5
2582/2582 [==============================] - 163s 63ms/step - loss: 0.5561 - accuracy: 0.8295 - val_loss: 0.5056 - val_accuracy: 0.8538

Epoch 00003: val_loss did not improve from 0.29193
Epoch 4/5
2582/2582 [==============================] - 161s 62ms/step - loss: 0.5222 - accuracy: 0.8482 - val_loss: 0.5631 - val_accuracy: 0.8576

Epoch 00004: val_loss did not improve from 0.29193
Epoch 5/5
2085/2582 [=======================>......] - ETA: 28s - loss: 0.5819 - accuracy: 0.8452